*********************************************************************************************************
# A Tour of Python 3
version 0.9 (alpha)

Authors: Phil Pfeiffer, Zack Bunch, and Feyi Oyeniyi<br>
East Tennessee State University<br>
Last updated February 2020<br>

*********************************************************************************************************

# Contents <a name='Contents'></a>
<br>10. [OO programming in Python](#OO-Programming-In-Python) <br>
&ensp; 10.1 [ Encapsulation and data hiding](#OO-Programming-In-Python-Encapsulation-And-Data-Hiding) <br>
&ensp; 10.2 [Dependency inversion](#OO-Programming-In-Python-Dependency-Inversion) <br>
&ensp; 10.3 [Doctest](#OO-Programming-In-Python-Doctest) <br>
&ensp; 10.4 [Nested classes](#OO-Programming-In-Python-Nested-Classes)

# 10.  OO programming in Python <a name='OO-Programming-In-Python'></a>


## 10.1  Encapsulation and data hiding <a name='OO-Programming-In-Python-Encapsulation-And-Data-Hiding'></a>
Like other OO languages, Python supports encapsulation.  An object B that an object A contains must, by default, be referenced "through" A, using an expression like `A.B`. The one exception to this rule involves the use multi-part `import` statements, like *from A import B*. 

Unlike compiled OO languages, Python lacks support for strong data hiding.  Declarations like "public", "private" and "protected" that are standard for compiled OO languages are absent from Python. To put this point in another way, **data hiding in Python is a programmer-enforced convention**. With the exception of a few built-in immutable classes and constants, all Python identifiers are ultimately readable and updateable, as are the objects they reference. The standard rationale for this lack of checking is threefold:
-  Enforcing data hiding in the absence of static type-checking would slow every data access.    Most of these checks, moreover, would be useless and redundant.
-  Treating data hiding as a convention rather than a requirement gives programmers freedom to break data hiding at need.
-  Ultimately, given the expressive power of constructs like `eval` and `exec`, enforcing data hiding may be a lost cause.

Python does support one weak mechanism for data hiding.  This mechanism, *name mangling*, is invoked automatically for
attributes whose names start with a double underscore.  Python manages these attributes by
-  assigning them new names and
-  converting all references to these attributes from within their container objects to these new names

This transforming of attribute names effectively invalidates all external references to the "real" names.  This transformation, however, is easily circumvented.

In [ ]:
# 10.1.a  'weak' data hiding using data mangling

class MyClass:
  def __init__(self, v):  self.__value = v
  def get_value(self):    return self.__value

instance = MyClass(1)
print( 'result of using get_value() to retrieve value from instance: ', instance.get_value() ) 

print( 'result of reading instance.__value: ', end='' )
try:
  print( instance.__value )
except:
  print( 'failure' )

print( 'result of setting instance.__value to 4, then using instance.get_value() to read it: ', end='' )
try:
  instance.__value = 4
  print( instance.get_value() )
except:
  print( 'failure' )

print()
print( 'result of reading instance._MyClass__value: ', end='' )
try:
  print( instance._MyClass__value )
except:
  print( 'failure' )

print( 'result of setting instance._MyClass__value to 4, then using instance.get_value() to read it: ', end='' )
try:
  instance._MyClass__value = 4
  print( instance.get_value() )
except:
  print( 'failure' )

##  10.2 Dependency inversion <a name='OO-Programming-In-Python-Dependency-Inversion'></a>
The term "dependency inversion" refers to the structuring of a family of modules as codes with a common interface. In compiled languages, type-related considerations require that the common interface be defined as a base class, which is then specialized to address the needs of particular realizations of that interface.  A classic explanation for the approach posits a base class that returns a time of day, coupled with a set of child classes that retrieve time-of-day information from different kinds of timepieces, ranging from digital assistants to digital clocks to sundials.

The term "dependency inversion" was coined at a time when objects first started to supplant procedures as the primary basis for software design.  The approach was developed as an alternative to *run-time type inference* (RTTI):  a design strategy that
-  treats related modules that have different preconditions for operation as different entities, and
-  selects which module to call at run-time via a series of tests that assess the current execution context.
RTTI is now viewed as an awful design practice, because it potentially hinders program evolution. In RTTI-based codes, adding modules to or removing modules from a given family of modules can force changes to *all* codes that access *any* of these modules.  Dependency inversion, by contrast, localizes such changes to the modules proper, so long as those modules' common interfaces can be kept constant.

In interpreted languages, the lack of compile-time type checking eliminates the need to define a base class in order to assure the child class's type consistency with their context of use.  So long as a class has attributes that fit a given context, their parent object's type is assumed to be acceptable.  This approach to type checking is referred to as *latent* typing or *duck* typing:  i.e., if an object walks like a duck and quacks like a duck, assume it's a duck.

Latent typing is favored by a minimalist school of software design, including authorities like Russ Olsen (<u>Eloquent Ruby</u>) who argue for achieving software quality by
-  writing concise programs with clearly named variables
-  supporting them with carefully crafted regression test suites.

These authorities criticize type declarations and hard-coded type checks as syntactic clutter:  constructs that attempt to prevent potential errors that would be better prevented by careful attention to naming and regression testing.  While these authorities promote inheritance as a means of eliminating duplicate code, they see it as useless for defining types.

Additional Python constructs used in these examples:
- `ABCMeta` - a Python *metaclass* that prevents a class from being used to instantiate other classes; the constrained   class is similar to abstract base classes in compiled OO languages.

Some useful terminology:
- *metaclass* - a class that controls the operation of a class.  All classes have metaclasses: the default is `object`.   Programming with metaclasses is something of a specialty in its own right; for more information, see (e.g.) 
  - Mark Lutz's <u>Learning Python</u>, 5th edition
  - Mark Summerfield's <u>Python3</u>

These examples use the following Python library resources:
-  `input` - read a string from the user's terminal
-  `time.strftime` - display time of day as a string


In [ ]:
# 10.2.a   An RTTI-based code (bad!) that
# -  solicits an "option 1 or 2" answer from a user in one of several languages, then
# -  launches a command in response to this response

# supporting constants
TIME_AS_24_HR_FORMAT = '%H:%M'      # time.strftime string for military time format
TIME_AS_12_HR_FORMAT = '%I:%M %p'   # time.strftime string for am/pm time format
OPTSTRINGS = {'1': TIME_AS_24_HR_FORMAT, '2': TIME_AS_12_HR_FORMAT}

# supporting library modules
import time

def QueryInEnglish():
  while True:
    option = input('Please enter 1 for 24 hour format, 2 for AM/PM format: ')
    if option in OPTSTRINGS.keys(): return option

def QueryInFrench():
  while True:
    option = input('S\'il vous plaît entrer 1 pour le format de 24 heures, 2 pour le format AM / PM: ')
    if option in OPTSTRINGS.keys(): return option

def QueryInSpanish():
  while True:
    option = input('Por favor, introduzca 1 para el formato de 24 horas, 2 para el formato AM / PM: ')
    if option in OPTSTRINGS.keys(): return option

def get_time(language):
  if language=="EN":
    return time.strftime(OPTSTRINGS[QueryInEnglish()])
  elif language=="FR":
    return time.strftime(OPTSTRINGS[QueryInFrench()])
  elif language=="ES":
    return time.strftime(OPTSTRINGS[QueryInSpanish()])
  else:
    return f'language not supported: {language}' 

print( 'Time in English is', get_time("EN"), '\n' )
print( 'Time in French is',  get_time("FR"), '\n' )
print( 'Time in Spanish is', get_time("ES"), '\n' )
print( 'Time in Italian is', get_time("IT") )

In [ ]:
# 10.2.b   redoing the previous example using dependency inversion
#  the following code encapsulates queries as classes with an abstract base class,
#  and features a type check for the get_time() routine parameter
#
#  TwoOptionQuery metaclasses are classes that control the instantiation of classes
#  for more on metaclasses, see the resources described at the outset of this document

# supporting constants
TIME_AS_24_HR_FORMAT = '%H:%M'      # time.strftime string for military time format
TIME_AS_12_HR_FORMAT = '%I:%M %p'   # time.strftime string for am/pm time format
OPTSTRINGS = {'1': TIME_AS_24_HR_FORMAT, '2': TIME_AS_12_HR_FORMAT}

# supporting library modules
import time
from abc import ABCMeta

# supporting functions
make_printable = lambda exception: '' if str(exception) is None else str(exception)

class TwoOptionQuery(metaclass=ABCMeta):
  @classmethod
  def getOption(clas):
    while True:
      option = input(clas.querystring)
      if option in OPTSTRINGS.keys(): return option

class QueryInEnglish(TwoOptionQuery):
  querystring = 'Please enter 1 for 24 hour format, 2 for AM/PM format: '

class QueryInFrench(TwoOptionQuery):
  querystring = 'S\'il vous plaît entrer 1 pour le format de 24 heures, 2 pour le format AM / PM: '

class QueryInSpanish(TwoOptionQuery):
  querystring = 'Por favor, introduzca 1 para el formato de 24 horas, 2 para el formato AM / PM: '

class QueryInItalian:
  @staticmethod
  def getOption():
    while True:
      option = input( 'Inserisci 1 per il formato 24 ore, 2 per il formato AM / PM: ' )
      if option in OPTSTRINGS.keys(): return option

def get_time(querier):
  # the following check would be automatic in compiled OO languages 
  # in Python, it's optional - and included for the sake of completeness
  assert issubclass(querier, TwoOptionQuery), f'querier ({querier.__name__}) must be a subclass of TwoOptionQuery' 
  return time.strftime({'1': TIME_AS_24_HR_FORMAT, '2': TIME_AS_12_HR_FORMAT}[querier.getOption()])

print( 'Time in English is', get_time(QueryInEnglish), '\n' )
print( 'Time in French is',  get_time(QueryInFrench), '\n' )
print( 'Time in Spanish is', get_time(QueryInSpanish), '\n' )
try:
  print( 'Time in Italian is' )
  print( get_time(QueryInItalian) )
except Exception as exception:
  print( '??',make_printable(exception) )

print( '\n\nHowever, the check in get_time can be bypassed by invoking the subclass directly\n' )
time.strftime({'1': "%H:%M", '2': "%I:%M %p"}[QueryInItalian.getOption()])

In [ ]:
# 10.2.c  simplifying the previous example, per "duck typing" practice.
#  -.  the TwoOptionQuery superclass is retained, because it provides common implementation logic
#      for its child classes.
#  -.  the ABCMeta designation has been discarded, however, as the assertion in get_time

# supporting constants
TIME_AS_24_HR_FORMAT = '%H:%M'      # time.strftime string for military time format
TIME_AS_12_HR_FORMAT = '%I:%M %p'   # time.strftime string for am/pm time format
OPTSTRINGS = {'1': TIME_AS_24_HR_FORMAT, '2': TIME_AS_12_HR_FORMAT}

# supporting library modules
import time

class TwoOptionQuery:
  @classmethod
  def getOption(clas):
    while True:
      option = input(clas.querystring)
      if option in OPTSTRINGS.keys(): return option

class QueryInEnglish(TwoOptionQuery):
  querystring = 'Please enter 1 for 24 hour format, 2 for AM/PM format: '

class QueryInFrench(TwoOptionQuery):
  querystring = 'S\'il vous plaît entrer 1 pour le format de 24 heures, 2 pour le format AM / PM: '

class QueryInSpanish(TwoOptionQuery):
  querystring = 'Por favor, introduzca 1 para el formato de 24 horas, 2 para el formato AM / PM: '

class QueryInItalian:
  @staticmethod
  def getOption():
    while True:
      option = input('Inserisci 1 per il formato 24 ore, 2 per il formato AM / PM: ')
      if option in OPTSTRINGS.keys(): return option

def get_time(querier):
  return time.strftime(OPTSTRINGS[querier.getOption()])


print( 'Time in English is', get_time(QueryInEnglish) )
print( 'Time in French is',  get_time(QueryInFrench) )
print( 'Time in Spanish is', get_time(QueryInSpanish) )
print( 'Time in Italian is', get_time(QueryInItalian) )

In addition to `ABCMeta`, the Python library provides decorators that treat class's methods and properties as abstract methods and properties: i.e., objects that the class's subclasses are required to overload.  See the Python Library documentation on [abstract base classes for containers](https://docs.python.org/3/library/collections.abc.html) for details.


## 10.3 Doctest  <a name='OO-Programming-In-Python-Doctest'></a>
Docstrings have a special significance for the Python library's [doctest](https://docs.python.org/3/library/doctest.html) module.  This module's `run` method
-  searches docstrings for substrings that are formatted as docstring-like test cases
    -  Test cases are signaled with initial ">>>" prompt strings.
-  executes those commands to confirm that they return the specified results and/or have the desired effects.
    -  Expected results are given after the commands to execute.
    -  "Traceback" results are treated specially:        ..., with the ELLIPSIS option enabled, causes doctest to ignore Traceback details when checking expected results.

The Python library manual's documentation on `doctest` gives further examples that show how to run `doctest` on entire modules-- i.e., .py files-- from the command line as well as from Python codes: a mode of use that the manual says is much more common in practice.

In [ ]:
# 10.3   A sample routine for generating powers of 2, with doctest code.

def powers_of_2():
  """generate successive powers of 2, starting with 2^0 """
  current_power_of_2 = 1
  while True:
    yield current_power_of_2
    current_power_of_2 *= 2

def print_first_n_powers_of_2(n):
  """  print the first n powers of 2, starting with 2^0

  routine prints the first n powers of 2 for a user-supplied integer n,
  starting with 2^0 and ending with 2^n-1.

  >>> print_first_n_powers_of_2(0.5)
  Traceback (most recent call last):
     ...
  TypeError: 'float' object cannot be interpreted as an integer
  >>> print_first_n_powers_of_2(-1)
  >>> print_first_n_powers_of_2(0)
  >>> print_first_n_powers_of_2(1)
  1
  >>> print_first_n_powers_of_2(5)
  1
  2
  4
  8
  16
  """
  p2 = powers_of_2()           # obtain a copy of the generator function for local use
  for i in range(0,n):  print(next(p2))

import doctest

# Show all test cases and their results as the cases execute
doctest.run_docstring_examples(print_first_n_powers_of_2, None, optionflags=doctest.ELLIPSIS, verbose=True)

# Limit output to failed tests (default)
doctest.run_docstring_examples(print_first_n_powers_of_2, None, optionflags=doctest.ELLIPSIS)

Doctest, which is notable for its declarative approach to testing, is being superseded by `pytest`, a standalone 
utility for declarative unit testing. Still, the tool illustrates what can be done to use comments to drive testing.

##  10.4 Nested classes  <a name='OO-Programming-In-Python-Nested-Classes'></a>
Python supports the nesting of classes within classes.  While nesting is unusual, the practice can be used 
to express the dependence of one class on another

In [ ]:
# 10.4   A representation of a common-- though far from the only-- ranking of cards in card decks.
# For simplicity, the design assumes that the ranking is uniform by suit - 
# an assumption that fails to hold for (e.g.) euchre-family card games.

class CardValues(object):
    def __init__(self, values = ['ace', 'king', 'queen', 'jack', '10', '9', '8', '7', '6', '5', '4', '3', '2']):
        self.vals = values
    def __len__(self):
        return len(self.vals)
    def __getitem__(self, i):
        return self.vals[i]
    def __eq__(self, other):
        return isinstance(other, CardValues) and len(self) == len(other) and all([self[i] == other[i] for i in range(0, len(self))])
    def values(self):  return self.vals
    def outranks(self, v1, v2):
        assert v1 in self.vals, f'value ({v1}) not in values ({self.vals})' 
        assert v2 in self.vals, f'value ({v2}) not in values ({self.vals})' 
        return self.vals.index(v1) < self.vals.index(v2)

class CardSuits(object):
    def __init__(self, suits = ['spades', 'hearts', 'diamonds', 'clubs']):
        self.suit_names = suits
    def __len__(self):
        return len(self.suit_names)
    def __getitem__(self, i):
        return self.suit_names[i]
    def __eq__(self, other):
        return isinstance(other, CardSuits) and len(self) == len(other) and all([self[i] == other[i] for i in range(0, len(self))])
    def suits(self):  return self.suit_names
    def outranks(self, s1, s2):
        assert s1 in self.suit_names, f'suit ({s1}) not in suits ({self.suit_names})' 
        assert s2 in self.suit_names, f'suit ({s2}) not in suits ({self.suit_names})' 
        return self.suit_names.index(s1) < self.suit_names.index(s2)

class CardDeck(CardValues, CardSuits):
    class Card(object):
        def __init__(self, deck, suit, value):
            self.deck, self.suit, self.value = deck, suit, value
        def comparable(self, other):
            return isinstance(other, CardDeck.Card) and self.deck.suits() == other.deck.suits() and self.deck.values() == other.deck.values()
        def __eq__(self, other):
            return self.comparable(other) and self.suit == other.suit and self.value == other.value
        def __gt__(self, other):
            return self.comparable(other) and deck.outranks(self, other)
        def __lt__(self, other):
            return self.comparable(other) and deck.outranks(other, self)
    def __init__(self, values=None, suits=None):
        if values == None:  CardValues.__init__(self)
        else:               CardValues.__init__(self, values)
        if suits == None:   CardSuits.__init__(self)
        else:               CardSuits.__init__(self, suits)
    def isSuit(self, suit):    return suit in self.suits()
    def isValue(self, value):  return value in self.values()
    def getCard(self, suit, value):
        assert self.isSuit(suit),    f"suit  ({suit}) missing from deck's suits  ({self.suits()})" 
        assert self.isValue(value),  f"value ({value}) missing from deck's values ({self.values()})" 
        return self.Card(self, suit, value)       # can also be CardDeck.Card
    def __eq__(self, other):
        return isinstance(other, CardDeck) and self.suits() == other.suits() and self.values() == other.values()
    def outranks(self, card1, card2):
        assert isinstance(card1, CardDeck.Card),  f'card ({card1}) missing from deck' 
        assert isinstance(card2, CardDeck.Card),  f'card ({card2}) missing from deck' 
        return card1.suit == card2.suit and CardValues.outranks(self, card1.value, card2.value)

# some operations on cards...

compare_cards = lambda c1, c2: 'outranks' if c1 > c2 else ( 'outranked by' if c1 < c2 else 'is incomparable to' )

deck = CardDeck()
ace_spades = deck.getCard('spades', 'ace')
deuce_spades = deck.getCard('spades', '2')
ace_clubs = deck.getCard('clubs', 'ace')
print( f'The ace of spades {compare_cards( ace_spades, deuce_spades )} the 2 of spades'  )
print( f'The ace of spades {compare_cards( ace_spades, ace_clubs )} the ace of clubs'  )
print( f'The 2 of spades {compare_cards( deuce_spades, ace_clubs )} the ace of clubs'  )

**Exercise**:
-  Modify the previous example so that ranking is determined by card within suit; i.e., so that all spades outrank all clubs.